<a href="https://colab.research.google.com/github/santomon/cv2021_4/blob/master/CV2021_ps4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Vision

---

**Goethe University Frankfurt am Main**

Winter Semester 2021/22

<br>

## *Problem Set 4*

---

**Points:** 200<br>
**Due:** 21. December 2021, 10am<br>
**Contact:** Matthias Fulde ([fulde@cvai.cs.uni-frankfurt.de](mailto:fulde@cvai.cs.uni-frankfurt.de))

---

**Name:** *Please write your name here.*

<br>

## Notes

---

Before you submit your solutions, please make sure that you

- wrote your name in the **Name** field above.
- wrote your code only into the designated areas delimited by `START OF YOUR CODE` and `END OF YOUR CODE` comments.
- removed all test code and test outputs you added during development.
- executed all code cells that generate output without error.

Your submission should be an archive that contains all files and folders provided with this notebook, but **not** the dataset.

<br>


## Table of Contents

---

- [1 Layers](#1-Layers-(160-Points))
  - [1.1 Vectorization Layer](#1.1-Vectorization-Layer-(5-Points))
  - [1.2 Linear Layer](#1.2-Linear-Layer-(35-Points))
  - [1.3 Dropout Layer](#1.3-Dropout-Layer-(25-Points))
  - [1.4 Convolutional Layer](#1.4-Convolutional-Layer-(50-Points))
  - [1.5 Max Pooling Layer](#1.5-Max-Pooling-Layer-(35-Points))
  - [1.6 ReLU Activation Layer](#1.6-ReLU-Activation-Layer-(10-Points))
- [2 Loss Function](#2-Loss-Function-(20-Points))
  - [2.1 Cross-entropy Loss](#2.1-Cross-entropy-Loss-(20-Points))
- [3 Optimization](#3-Optimization-(10-Points))
  - [3.1 SGD with Momentum](#3.1-SGD-with-Momentum-(10-Points))
- [4 Deep Neural Network](#4-Deep-Neural-Network-(10-Points))
  - [4.1 Training](#4.1-Training-(10-Points))

<br>

## Setup

---

In this problem set we **only** work with NumPy and Matplotlib.

We set the Matplotlib backend to inline in order to display images directly in the notebook. In addition, we load all modules containing the classes we want to implement in this problem set and enable autoreloading, so that we don't have to reload the modules manually when we've made an edit.

For this problem set, we recommend to use at least Python 3.7.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from modules import *

%load_ext autoreload
%autoreload 2

## Definitions

---

We define an `error` function to measure the relative difference between two outcomes.

In [ ]:
def error(x, y):
    """
    Calculate the sum of the relative differences.
    The absolute differences are scaled with the sum of the absolute values.
    """
    x = x.astype(np.float32)
    y = y.astype(np.float32)

    return np.sum(abs(x - y) / np.maximum(1e-8, abs(x) + abs(y)))

## Exercises

---

### 1 Layers (160 Points)

---

Our goal in this problem set is to implement a deep neural network composed of different kinds of layers.

The first part of the network should be a CNN, composed of convolutional and max pooling layers, and the second part should be a FCN with linear layers, for which we also want to implement dropout for regularization. We're going to use standard rectified linear units as nonlinear activation functions for both the convolutional and the linear layers.

In this exercise we implement the different layers so that we can easily plug them into the model later.

<br>

### 1.1 Vectorization Layer (5 Points)

---

Since we want to use linear layers, we need a function to convert tensor input into vectors. Moreover, since we want to implement a small FCN on top of a CNN, we also need a function to reverse this operation, in order to pass the gradient of the loss with the correct shape to the last convolutional layer.

##### Task

Complete the definition of the `Vector` class in the `layers/vector.py` file. Your implementation should be fully vectorized, so no loops are allowed.

In the `forward` method, store the shape of the input to be used in the backward pass. Convert the inputs into vectors such that the result is a matrix where each row is one input. Store the result in the `outputs` variable that is returned from the method.

In the `backward` method, apply the revesed operation and restore the original shape for the gradient of the loss that the method receives from the following layer. Store the result in the `in_grad` variable that is returned from the method.

##### Results

To test your implementation, run the following code cells.

In [ ]:
# Set shape for test inputs and outputs.
inputs_shape  = (2, 3, 4, 5)
outputs_shape = (2, 60)

# Create test inputs.
inputs = np.zeros(inputs_shape)

# Create upstream gradient.
out_grad = np.zeros(outputs_shape)

# Create layer.
layer = Vector()

You can test your `forward` method with the following code.

In [ ]:
# Compute forward pass.
outputs = layer.forward(inputs)

# Compare shapes
print(f'Forward method correct: {outputs.shape == outputs_shape}')

You can test your `backward` method with the following code.

In [ ]:
# Compute forward pass.
outputs = layer.forward(inputs)

# Compute backward pass.
in_grad = layer.backward(out_grad)

# Compare shapes.
print(f'Backward method correct: {in_grad.shape == inputs_shape}')

### 1.2 Linear Layer (35 Points)

---

The fully-connected network is composed of linear or affine layers that map input vectors with shape $(\text{num_samples},\text{in_features})$ to output vectors with shape $(\text{num_samples},\text{out_features})$, where the difference between a linear and an affine map is whether a bias is added or not. In this exercise we want to implement such a layer as described below.

##### Task

Complete the definition of `Linear` class in the `layers/linear.py` file. Your implementation should be fully vectorized, that is, no loops are allowed.

In the `__init__` method, store a flag indicating if a linear or affine transformation should be used.<br>
Initialize the parameters with random values as follows:<br>

- The parameters are stored in a dictionary already created in the base class. Save the weights and, conditionally, the bias, as in
  ```python
  self.param['weights'] = ...
  self.param['bias'] = ...
  ```
- The weights should have the shape $(\text{in_features},\text{out_features})$ and the bias should have the shape $(\text{out_features})$, where $\text{in_features}$ is the length of the input vectors and $\text{out_features}$ is the length of the output vectors.
- Initialize each parameter value from $\text{Uniform}(-\sqrt{k},\sqrt{k})$ where $k = \frac{1}{\text{in_features}}$.
   
<br>

In the `forward` method, store the received inputs for gradient computation in the backward pass. Depending on the stored flag, apply a linear or affine transformation to the inputs and store the result in the `outputs` variable that is returned from the method.
   
<br>

In the `backward` method, compute the gradient of the loss with respect to the parameters and inputs. The method receives the gradient of the loss with respect to the layer output.
   
- The gradient of the loss with respect to the weights and, if required, the bias, is stored in a dictionary inherited from the base class. Save the parameters using the same keys that were used for the parameter dictionary, as in
  ```python
  self.grad['weights'] = ...
  self.grad['bias'] = ...
  ```
- Store the gradient of the loss with respect to the layer inputs in the `in_grad` variable that is returned from the method.

##### Results

To test your implementation, run the following code cells.

In [ ]:
# Set number of samples.
num_inputs = 2

# Set input and output dimensions.
input_dim = 5
output_dim = 3

# Create inputs.
inputs = np.linspace(-0.1, 0.5, num=num_inputs * input_dim)
inputs = np.reshape(inputs, (num_inputs, input_dim))

# Create weights.
weights = np.linspace(-0.2, 0.3, num=output_dim * input_dim)
weights = np.reshape(weights, (input_dim, output_dim))

# Create bias.
bias = np.linspace(-0.3, 0.1, num=output_dim)

# Create upstream gradient.
out_grad = np.linspace(-0.2, 0.4, num=num_inputs * output_dim)
out_grad = np.reshape(out_grad, (num_inputs, output_dim))

# Create affine layer.
layer = Linear(input_dim, output_dim)

# Reset parameters.
layer.param['weights'] = weights
layer.param['bias'] = bias

You can test your `forward` method with the following code.

In [ ]:
# Compute forward pass.
outputs = layer.forward(inputs)

correct_outputs = np.array([
    [-0.22619048, -0.0202381,   0.18571429],
    [-0.20238095,  0.06309524,  0.32857143]
])

# Compare outputs.
print(f'Forward method correct: {error(outputs, correct_outputs) < 1e-5}')

You can test your `backward` method with the following code.

In [ ]:
# Compute forward pass.
outputs = layer.forward(inputs)

# Compute backward pass.
in_grad = layer.backward(out_grad)

# Get gradients for parameters.
weights_grad = layer.grad['weights']
bias_grad = layer.grad['bias']

# Compare gradients.
results = [

    error(in_grad, np.array([
        [ 0.048,       0.02228571, -0.00342857, -0.02914286, -0.05485714],
        [-0.12942857, -0.03942857,  0.05057143,  0.14057143,  0.23057143]
    ])),
    
    error(weights_grad, np.array([
        [0.05733333, 0.07333333, 0.08933333],
        [0.05466667, 0.08666667, 0.11866667],
        [0.052,      0.1,        0.148     ],
        [0.04933333, 0.11333333, 0.17733333],
        [0.04666667, 0.12666667, 0.20666667]
    ])),
    
    error(bias_grad, np.array([-0.04, 0.2, 0.44]))
]

# Show results.
print(f'Backward method correct: {all(e < 1e-5 for e in results)}')

### 1.3 Dropout Layer (25 Points)

---

In the lecture, dropout was introduced as a regularization method. Applied to the activations of a linear layer, a random binary mask is created such that each activation unit is either kept with probability $p$, or set to zero with probability $1 - p$. Thus, in every training iteration, a different subset of neurons is learned, forcing the network to learn redundant feature representations.

##### Task

Complete the definition of the `Dropout` class in the `layers/dropout.py` file. Your implementation should be fully vectorized, so no loops are allowed.

<br>

In the `forward` method, apply dropout when the `self.training` attribute is true. Otherwise just pass the input. For the implementation of dropout

- create a random binary mask with the same shape as the input, such that each entry is $1$ with probability $p$ and $0$ with probability $1 - p$.
- scale the mask with $\frac{1}{p}$ so that we don't have to scale the activations during inference, where all neurons should be kept active.
- multiply each input feature with the corresponding entry in the mask.
- store the mask for gradient computation in the backward pass.

Store the transformed activations in the `outputs` variable that is returned by the method.

<br>

In the `backward` method, compute the gradient of the loss with respect to the input. The local gradient of the dropout layer with respect to the input is just the random mask created in the forward pass. This must be combined with the gradient received from the next layer. Store the result in the `in_grad` variable that is returned by the method.

##### Results

To test your implementation, run the following code cells.

In [ ]:
# Create test inputs.
inputs = np.random.randn(1000, 1000) + 10

# Create upstream gradient.
out_grad = np.random.randn(1000, 1000) + 10

# Create candidate probabilities.
probs = [0.3, 0.5, 0.7]

You can test your `forward` method with the following code.

In [ ]:
# Create list for results.
results = []

# Compute forward passes.
for p in probs:
    layer = Dropout(p)

    # Get fraction of dropped neurons during training.
    outputs = layer.forward(inputs)
    train_result = np.around(np.mean(outputs == 0), decimals=1) + p

    layer.training = False

    # Get fraction of dropped neurons during testing.
    outputs = layer.forward(inputs)
    test_result = np.around(np.mean(outputs == 0), decimals=1)
    
    # Store results.
    results.append((train_result, test_result))

# Compare results.
print(f'Forward method correct: {all(res == (1, 0) for res in results)}')

You can test your `backward` method with the following code.

In [ ]:
# Create layer.
layer = Dropout(0.5)

# Compute forward pass.
outputs = layer.forward(inputs)

# Compute backward pass.
in_grad = layer.backward(out_grad)

# Compare results.
print(f'Backward method correct: {np.array_equal(np.where(outputs == 0), np.where(in_grad == 0))}')

### 1.4 Convolutional Layer (50 Points)

---

For the CNN part of our model we need convolutional layers. For inputs with shape $(\text{num_samples},\text{in_channels},\text{in_height},\text{in_width})$ the layer should generate output with shape $(\text{num_samples},\text{out_channels},\text{out_height},\text{out_width})$, with the spacial dimensions of the output depending on the size of the input, the kernel size and the padding and stride applied in the convolution.

##### Task

Complete the definition of `Conv2D` class in the `layers/conv.py` file. Your implementation should be at least partly vectorized, that is, you're allowed to use at most *four* loops each for the forward and backward pass.

In the `__init__` method, store the given values for padding and stride, and a flag indicating if a bias should be used or not.<br>
Initialize the parameters with random values as follows:<br>

- The parameters are stored in a dictionary already created in the base class. Save the weights and, conditionally, the bias, as in
  ```python
  self.param['weights'] = ...
  self.param['bias'] = ...
  ```
- The weights should have the shape $(\text{out_channels},\text{in_channels},\text{kernel_size},\text{kernel_size})$ and the bias should have the shape $(\text{out_channels})$, thus you can assume that the filters are always square.
- Initialize each parameter value from $\text{Uniform}(-\sqrt{k},\sqrt{k})$ where $k = \frac{1}{\text{in_channels} \:\cdot\: \text{kernel_size}^2}$.
   
<br>

In the `forward` method, store the received inputs for gradient computation in the backward pass. Convolve the inputs with the filters using the stored values for padding and stride and conditionally add the bias, then store the result in the `outputs` variable that is returned from the method.
   
<br>

In the `backward` method, compute the gradient of the loss with respect to the parameters and inputs. The method receives the gradient of the loss with respect to the layer output.
   
- The gradient of the loss with respect to the weights and, if required, the bias, is stored in a dictionary inherited from the base class. Save the parameters using the same keys that were used for the parameter dictionary, as in
  ```python
  self.grad['weights'] = ...
  self.grad['bias'] = ...
  ```
- Store the gradient of the loss with respect to the layer inputs in the `in_grad` variable that is returned from the method.

##### Results

To test your implementation, run the following code cells.

In [ ]:
# Set dimensions for inputs, outputs and weights.
inputs_shape  = (2, 2, 3, 3)
outputs_shape = (2, 2, 2, 2)
weights_shape = (2, 2, 3, 3)

# Create inputs.
inputs = np.linspace(-0.1, 0.5, num=np.prod(inputs_shape))
inputs = np.reshape(inputs, inputs_shape)

# Create weights.
weights = np.linspace(-0.2, 0.3, num=np.prod(weights_shape))
weights = np.reshape(weights, weights_shape)

# Create bias.
bias = np.linspace(-0.1, 0.2, num=3)

# Create upstream gradient.
out_grad = np.linspace(-0.2, 0.3, num=np.prod(outputs_shape))
out_grad = np.reshape(out_grad, outputs_shape)

# Create conv layer.
layer = Conv2D(2, 2, kernel_size=3, padding=1, stride=2)

# Reset parameters.
layer.param['weights'] = weights
layer.param['bias'] = bias

You can test your `forward` method with the following code.

In [ ]:
# Compute forward pass.
outputs = layer.forward(inputs)

correct_outputs = np.array([
    [[[-0.06,       -0.07012245],
      [-0.10212245, -0.124     ]],
     [[ 0.1135102,   0.13865306],
      [ 0.17718367,  0.19057143]]],
    [[[-0.18342857, -0.22881633],
      [-0.33134694, -0.3884898 ]],
     [[ 0.62485714,  0.61473469],
      [ 0.58273469,  0.56085714]]]
])

# Compare outputs
print(f'Forward method correct: {error(outputs, correct_outputs) < 1e-5}')

You can test your `backward` method with the following code.

In [ ]:
# Compute forward pass.
outputs = layer.forward(inputs)

# Compute backward pass.
in_grad = layer.backward(out_grad)

# Get gradients for parameters.
weights_grad = layer.grad['weights']
bias_grad = layer.grad['bias']

# Compare gradients.
results = [

    error(in_grad, np.array([
        [[[ 0.02095238,  0.04,        0.02      ],
          [ 0.03428571,  0.0647619,   0.03238095],
          [ 0.01904762,  0.03619048,  0.01809524]],
         [[-0.01333333, -0.02,       -0.00571429],
          [-0.01714286, -0.02095238, -0.00190476],
          [ 0.00190476,  0.01047619,  0.00952381]]],
        [[[ 0.01333333,  0.0247619,   0.01238095],
          [ 0.01904762,  0.03428571,  0.01714286],
          [ 0.01142857,  0.02095238,  0.01047619]],
         [[ 0.04761905,  0.10190476,  0.0552381 ],
          [ 0.1047619,   0.22285714,  0.12      ],
          [ 0.06285714,  0.13238095,  0.07047619]]]
    ])),

    error(weights_grad, np.array([
        [[[0.04933333, 0.09161905, 0.04114286],
          [0.08914286, 0.16419048, 0.0727619 ],
          [0.03295238, 0.05885714, 0.0247619 ]],
         [[0.05961905, 0.10190476, 0.04114286],
          [0.08914286, 0.14361905, 0.05219048],
          [0.02266667, 0.028,      0.00419048]]],
        [[[0.08209524, 0.15714286, 0.07390476],
          [0.15466667, 0.2952381,  0.13828571],
          [0.06571429, 0.12438095, 0.05752381]],
         [[0.13352381, 0.24971429, 0.11504762],
          [0.23695238, 0.4392381,  0.2       ],
          [0.09657143, 0.17580952, 0.07809524]]]
    ])),

    error(bias_grad, np.array([-0.13333333, 0.93333333, 0.]))

]

# Show results.
print(f'Backward method correct: {all(e < 1e-5 for e in results)}')

### 1.5 Max Pooling Layer (35 Points)

---

In order to reduce the size of the feature maps in the CNN, we want to use max pooling. For each channel separately, the input is filtered such that only the maximum activation in each filter position is selected.

##### Task

Complete the definition of `MaxPool` class in the `layers/pool.py` file. Your implementation should be at least partly vectorized, that is, you're allowed to use at most *four* loops each for the forward and backward pass.

In the `__init__` method, store the given values for kernel size and stride. If no stride is given, set the value to the kernel size, so that non-overlapping windows are used for pooling.
   
<br>

In the `forward` method, store the received inputs for gradient computation in the backward pass. Apply max pooling to the input using the given kernel size and stride, then store the result in the `outputs` variable that is returned from the method.
   
<br>

In the `backward` method, compute the gradient of the loss with respect to the inputs. The method receives the gradient of the loss with respect to the layer output. Store the gradient of the loss with respect to the layer inputs in the `in_grad` variable that is returned from the method.

##### Results

To test your implementation, run the following code cells.

In [ ]:
# Define shapes for input and output.
inputs_shape  = (2, 2, 4, 4)
outputs_shape = (2, 2, 2, 2)

# Create inputs.
inputs = np.linspace(-0.3, 0.5, num=np.prod(inputs_shape))
inputs = np.reshape(inputs, inputs_shape)

# Create upstream gradient.
out_grad = np.linspace(-0.2, 0.3, num=np.prod(outputs_shape))
out_grad = np.reshape(out_grad, outputs_shape)

# Create layer.
layer = MaxPool(2)

You can test your `forward` method with the following code.

In [ ]:
# Compute forward pass.
outputs = layer.forward(inputs)

correct_outputs = np.array([
    [[[-0.23650794, -0.21111111],
      [-0.13492063, -0.10952381]],
     [[-0.03333333, -0.00793651],
      [ 0.06825397,  0.09365079]]],
    [[[ 0.16984127,  0.1952381 ],
      [ 0.27142857,  0.2968254 ]],
     [[ 0.37301587,  0.3984127 ],
      [ 0.47460317,  0.5       ]]]
])

# Compare outputs.
print(f'Forward method correct: {error(outputs, correct_outputs) < 1e-5}')

You can test your `backward` method with the following code.

In [ ]:
# Compute forward pass.
outputs = layer.forward(inputs)

# Compute backward pass.
in_grad = layer.backward(out_grad)

correct_in_grad = np.array([
    [[[0.,  0.,         0.,  0.        ],
      [0., -0.2,        0., -0.16666667],
      [0.,  0.,         0.,  0.        ],
      [0., -0.13333333, 0., -0.1       ]],
     [[0.,  0.,         0.,  0.        ],
      [0., -0.06666667, 0., -0.03333333],
      [0.,  0.,         0.,  0.        ],
      [0.,  0.,         0.,  0.03333333]]],
    [[[0.,  0.,         0.,  0.        ],
      [0.,  0.06666667, 0.,  0.1       ],
      [0.,  0.,         0.,  0.        ],
      [0.,  0.13333333, 0.,  0.16666667]],
     [[0.,  0.,         0.,  0.        ],
      [0.,  0.2,        0.,  0.23333333],
      [0.,  0.,         0.,  0.        ],
      [0.,  0.26666667, 0.,  0.3       ]]]
])

# Compare results.
print(f'Backward method correct: {error(in_grad, correct_in_grad) < 1e-5}')

### 1.6 ReLU Activation Layer (10 Points)

---

For both the convolutional and fully-connected layers we want to use standard rectified linear units as activation function. The function is applied elementwise such that all positive values are left unchanged, while all negative values are set to zero.

##### Task

Complete the definition of the `ReLU` class in the `layers/relu.py` file. Your implementation should be fully vectorized, that is, no loops are allowed.

In the `forward` method, store the received inputs for gradient computation in the backward pass. Apply the ReLU activation function to the input componentwise and store the result in the `outputs` variable that is returned from the method.

In the `backward` method, compute the gradient of the loss with respect to the inputs. The method receives the gradient of the loss with respect to the layer output. Store the gradient of the loss with respect to the layer inputs in the `in_grad` variable that is returned from the method.

##### Results

To test your implementation, run the following code cells.

In [ ]:
# Create inputs.
inputs = np.array([
    [[[-0.01640701, -0.57770197],
      [ 0.37487488, -0.77260795]],
     [[ 0.31117251,  1.28778772],
      [-0.9739217,  -0.15076198]],
     [[ 0.67561644, -0.75053469],
      [-0.68843359,  1.13142207]]],
    [[[-0.48168604,  0.37160233],
      [ 0.41082495, -0.40195236]],
     [[-1.4123589,   2.57995339],
      [-0.20205541,  0.5211701 ]],
     [[ 0.36500362, -1.6598223 ],
      [ 3.39411579, -1.46596421]]]
])

# Create upstream gradient.
out_grad = np.linspace(-0.3, 0.4, num=np.prod(inputs.shape))
out_grad = np.reshape(out_grad, inputs.shape)

# Create layer.
layer = ReLU()

You can test your `forward` method with the following code.

In [ ]:
# Compute forward pass.
outputs = layer.forward(inputs)

correct_outputs = np.array([
    [[[0.,         0.        ],
      [0.37487488, 0.        ]],
     [[0.31117251, 1.28778772],
      [0.,         0.        ]],
     [[0.67561644, 0.        ],
      [0.,         1.13142207]]],
    [[[0.,         0.37160233],
      [0.41082495, 0.        ]],
     [[0.,         2.57995339],
      [0.,         0.5211701 ]],
     [[0.36500362, 0.        ],
      [3.39411579, 0.        ]]]
])

# Compare outputs.
print(f'Forward method correct: {error(outputs, correct_outputs) < 1e-5}')

You can test your `backward` method with the following code.

In [ ]:
# Compute forward pass.
outputs = layer.forward(inputs)

# Compute backward pass.
in_grad = layer.backward(out_grad)

correct_in_grad = np.array([
    [[[-0.,         -0.        ],
      [-0.23913043, -0.        ]],
     [[-0.17826087, -0.14782609],
      [-0.,         -0.        ]],
     [[-0.05652174, -0.        ],
      [ 0.,          0.03478261]]],
    [[[ 0.,          0.09565217],
      [ 0.12608696,  0.        ]],
     [[ 0.,          0.2173913 ],
      [ 0.,          0.27826087]],
     [[ 0.30869565,  0.        ],
      [ 0.36956522,  0.        ]]]
])

# Compare results.
print(f'Backward method correct: {error(in_grad, correct_in_grad) < 1e-5}')

### 2 Loss Function (20 Points)

---

For our model we want to use the cross-entropy loss that we already implemented in the previous problem set. However, this time we not only want to compute the gradient of the loss with respect to the input to the loss function, but with respect to all the parameters in our model. So we want to be able to call our loss function like this:

```python
# Outside training loop
loss = CrossEntropyLoss(model)

# Inside training loop
outputs = model(inputs)

current_loss = loss(outputs, labels)

loss.backward()
```

After calling the `backward` method on the loss, the gradients with respect to all the weights and biases should be stored in the `grad` dictionary of the respective layer, so that we can call the optimizer to actually perform the updates. Calling the model and loss directly as in the example code above is just a shortcut for calling the respective `forward` methods, which we implemented for convenience.

This is not a huge change compared to what we have already done, though.

The only difference is that we split the computation of the loss and the computation of the gradient into two methods, and instead of just returning the gradient with respect to the function's input, we also pass it to the `backward` method of the model, which we stored in the constructor of the loss.


### 2.1 Cross-entropy Loss (20 Points)

---

Now, let's recap the definition of the cross-entropy loss. For a single pair $(s^{(i)}, y_i)$ with input $s^{(i)}$ and target $y_i$, the cross-entropy loss is defined as

$$
    L(s^{(i)})
    =
    -\ln\frac{e^{s_{y_i}^{(i)}}}{\sum_{j=1}^K e^{s_j^{(i)}}}
    =
    -s_{y_i}^{(i)} + \ln\sum_{j=1}^K e^{s_j^{(i)}},
$$

where $s^{(i)} \in \mathbb{R}^K$ is a vector of scores for $K$ classes, which will be the output of our last network layer. The total loss for a set $S = \{s^{(i)}\}_{i=1}^N$ of $N$ examples is just the average of the losses for a single input, that is

$$
    \mathcal{L}(S) = \frac{1}{N} \sum_{i=1}^N L(s^{(i)}).
$$

The gradient of the loss $L$ for the $i$-th input with respect to the $k$-th entry is

<br>

$$
    \nabla L(s^{(i)})_k = \frac{\partial L(s^{(i)})}{\partial s_k^{(i)}}
    =
    \begin{cases}
        \sigma(s^{(i)})_k - 1 & \text{if} \enspace k = y_i \\[0.5em]
        \sigma(s^{(i)})_k & \text{if} \enspace k \neq y_i,
    \end{cases}
$$

where $\sigma$ denotes the softmax function, defined as

$$
    \sigma(s)_k = \frac{e^{s_k}}{\sum_{j=1}^K e^{s_j}}.
$$

Regarding the gradient of the total loss $\mathcal{L}$ with respect to the function inputs, we can use the linearity of the differential operator to obtain

$$
    \nabla \mathcal{L}(s^{(i)})_k = \frac{1}{N} \sum_{i=1}^N \nabla L(s^{(i)})_k.
$$

##### Task

Complete the definition of the `CrossEntropyLoss` class in the `loss.py` file. Your implementation should be fully vectorized, so no loops are allowed.

In the `forward` method, compute the total loss for the given set of inputs. Since we need the output of the softmax function both for the computation of the loss and the computation of the gradient, you should save this intermediate result for later use in the backward pass. Store the loss in the `outputs` variable that is returned from the method.

In the `backward` method, compute the gradient of the loss with respect to the input of the loss function and store the value in the `in_grad` variable. Then pass the gradient to the `backward` method of the model stored in the `self.model` attribute, to backpropagate the gradient through the entire network.

##### Results

To test your implementation, run the following code cells.

In [ ]:
# Create inputs.
inputs = np.array([
    [ 1.7, -2.5, -5.3, 4.1],
    [-1.2,  1.8,  2.0, 2.5],
    [ 2.3, -1.7, -0.4, 0.6]
])

# Create targets.
labels = np.array([0, 3, 2])

# Create dummy model.
model = Model()

# Create loss.
loss = CrossEntropyLoss(model)

You can test your `forward` method with the following code.

In [ ]:
# Compute forward pass.
test_loss = loss(inputs, labels)

correct_loss = 2.060289248015619

print(f'Forward method correct: {abs(test_loss - correct_loss) < 1e-5}')

You can test your `backward` method with the following code.

In [ ]:
# Compute forward pass.
loss(inputs, labels)

# Compute backward pass.
in_grad = loss.backward()

correct_in_grad = np.array([
    [-3.05645734e-01,  4.15191527e-04,  2.52478228e-05,  3.05205294e-01],
    [ 3.87302498e-03,  7.77917862e-02,  9.50151022e-02, -1.76679913e-01],
    [ 2.62838751e-01,  4.81405965e-03, -3.15669120e-01,  4.80163093e-02]
])

# Compare results.
print(f'Backward method correct: {error(in_grad, correct_in_grad) < 1e-5}')

### 3 Optimization (10 Points)

---

So far we have only used vanilla gradient descent. That is, the update rule was to scale the gradient with the learning rate and subtract the result from the parameters. In this exercise we want to extend that concept a bit, taking into account also the previous updates.

### 3.1 SGD with Momentum (10 Points)

---

One of the problems with standard gradient descent is that the gradient with respect to a parameter may change rapidly during training. These oscillations of the gradient make optimization hard. In addition, there is also the problem of the gradient being stuck in a flat region, where the slope is almost zero. Gradient descent with momentum is one approach to tackle these problems.

Instead of just updating the parameters with

$$
    W^{(t)} = W^{(t-1)} - \lambda\nabla\mathcal{L}(W^{(t-1)})
$$

where $\lambda$ is the learning rate, we also take into account the previous updates of the parameters, scaled by a hyperparameter called momentum. Thus the update rule becomes

$$
    W^{(t)} = W^{(t-1)} - V^{(t)}
$$

with

$$
    V^{(t)} = \lambda\nabla\mathcal{L}(W^{(t-1)}) + \mu V^{(t-1)}
$$

where $V$ is called velocity and $\mu$ is the momentum. The velocity is an array with the same shape as $W$ and $\nabla\mathcal{L}(W)$ and can be understood as a moving average over the past gradients. With this update rule we get a more stable trajectory towards a minimum and we can still move, even if the gradient for the current time step becomes small.

##### Task

Complete the definition of the `SGD` class in the `optim.py` file. Your implementation should be fully vectorized, so no loops are allowed.

In the `step` method of the optimizer, implement the update rule described above. The $\text{learning_rate}$ and $\text{momentum}$ are stored as attributes of the optimizer object and each layer that is iterated over has dictionaries for parameters, gradients and velocity, where corresponding entries are referenced with the same name, given that you adhered to this convention in the previous exercises.

##### Results

To test your implementation, run the following code cell.

In [ ]:
class Net(Model):

    def __init__(self):
        self.fc = Linear(3, 2, bias=False)

# Create dummy model.
model = Net()

# Set shape for weights, gradient and velocity.
shape = (3, 2)

# Create weights.
weights = np.linspace(-0.3, 0.5, num=np.prod(shape))
weights = np.reshape(weights, shape)

# Add weights to layer.
model.fc.param['weights'] = weights

# Create gradient.
grad = np.linspace(-0.2, 0.3, num=np.prod(shape))
grad = np.reshape(grad, shape)

# Add gradient to layer.
model.fc.grad['weights'] = grad

# Create optimizer.
optimizer = SGD(model, learning_rate=0.1, momentum=0.5)

# Create velocity.
velocity = np.linspace(-0.1, 0.4, num=np.prod(shape))
velocity = np.reshape(velocity, shape)

# Add velocity to layer.
model.fc.velocity['weights'] = velocity

# Compute update step.
optimizer.step()

correct_weights = np.array([
    [-0.23, -0.13],
    [-0.03,  0.07],
    [ 0.17,  0.27]
])

# Compare results.
print(f'Step method correct: {error(weights, correct_weights) < 1e-5}')

### 4 Deep Neural Network (10 Points)

---

Now that we have all the components implemented, we can plug everything together to create a deep neural network.

We want to train our model again on the CIFAR-10 dataset that we already used in the previous problem sets. The function for loading and preprocessing the data expects the dataset in the `datasets` folder in the same directory as the notebook, so copy the folder before you proceed.

In [ ]:
# Load and preprocess the CIFAR-10 dataset.
data = get_CIFAR_10_data()

# Output the shapes of the partitioned data and labels.
for name, array in data.items():
    print(f'{name} shape: {array.shape}')

Since we don't have GPU support, we're going to create a rather shallow model. Otherwise the training would take too much time.

To test our implementations, we're going to define a network with two blocks of convolutional layers, each followed by a ReLU activation function and a max pooling layer. We convert the outputs of the last pooling layer into vectors and pass them into a small fully-connected network, composed of two linear layers, the first of which has a ReLU activation function, followed by a dropout layer. The last linear layer has no activation function and produces the scores for the ten classes of the dataset.

Let's create this model using the `Model` base class.

In [ ]:
class Net(Model):
    
    def __init__(self):

        # Convolution with ReLU and max pool
        self.conv1 = Conv2D(3, 6, kernel_size=3, padding=1)
        self.relu1 = ReLU()
        self.pool1 = MaxPool(2)

        # Convolution with ReLU and max pool
        self.conv2 = Conv2D(6, 8, kernel_size=3, padding=1)
        self.relu2 = ReLU()
        self.pool2 = MaxPool(2)

        # Tensors to vector conversion
        self.vec = Vector()

        # Fully connected with ReLU
        self.fc1 = Linear(512, 32)
        self.relu3 = ReLU()

        # Dropout
        self.drop = Dropout(0.75)

        # Fully connected
        self.fc2 = Linear(32, 10)

### 4.1 Training (10 Points)

---

To train the model, we make use of the `Solver` class defined in the `solver.py` file.

##### Task

Train the model you implemented for at least $1$ epoch on the complete training set and show the results. If you use the default values of the solver, you should see a training and validation accuracy of at least $\sim 30 \%$ after one epoch. There should be no overfitting up to this point. In order to first check if your model converges, you can use the development set (`X_dev`, `y_dev`) from the `data` dictionary instead of the whole training set.

##### Results

You can use the `verbose` and `print_every` parameters of the solver to print out results during training.

In [ ]:
# Create the model.
model = Net()

# Define the training and validation set.
training_data = {
    'X_train': data['X_train'],
    'y_train': data['y_train'],
    'X_val': data['X_val'],
    'y_val': data['y_val']
}

# Create the solver.
solver = Solver(model, training_data, num_epochs=1, verbose=True)

# Train the model.
results = solver.train()